In [ ]:
%%capture

!pip install ratelimit skyfield

In [ ]:
LAT, LON = 20.994930509789185, 105.86682449422528
TIMEZONE = 'Asia/Ho_Chi_Minh'

STAR_HOUR, START_MINUTE, START_SECOND = 11, 4, 18
END_HOUR, END_MINUTE, END_SECOND = 12, 57, 48

In [ ]:
from google.colab import userdata

SPACE_TRACK_USERNAME = userdata.get('SPACE_TRACK_USERNAME')
SPACE_TRACK_PASSWORD = userdata.get('SPACE_TRACK_PASSWORD')

assert SPACE_TRACK_USERNAME, "Please add SPACE_TRACK_USERNAME in collab notebook"
assert SPACE_TRACK_PASSWORD, "Please add SPACE_TRACK_PASSWORD in collab notebook"

In [ ]:
%%capture

import os
from google.colab import drive

drive.mount('/content/drive')

OMM_DIR = '/content/drive/MyDrive/GNSS-OMM'
if not os.path.exists(OMM_DIR):
    os.makedirs(OMM_DIR)

In [ ]:
import requests
import os

from ratelimit import limits, sleep_and_retry

QZSS_IDS = [
  42738,
  42917,
  42965,
  49336,
  62876,
]

GLONASS_IDS = [
  32275,
  32276,
  32393,
  32395,
  36111,
  36112,
  36402,
  37867,
  37868,
  37869,
  39155,
  39620,
  40001,
  40315,
  41330,
  42939,
  43508,
  43687,
  44299,
  44850,
  45358,
  46805,
  52984,
  54031,
  54377,
  57517,
]

GALILEO_IDS = [
  37846,
  37847,
  38857,
  40128,
  40129,
  40544,
  40545,
  40889,
  40890,
  41174,
  41175,
  41549,
  41550,
  41859,
  41860,
  41861,
  41862,
  43055,
  43056,
  43057,
  43058,
  43564,
  43565,
  43566,
  43567,
]

GPS_IDS = [
  24876,
  25030,
  25933,
  26360,
  26407,
  26605,
  26690,
  27663,
  28129,
  28190,
  28361,
  28474,
  28874,
  29486,
  29601,
  32260,
  32384,
  32711,
  34661,
  35752,
  36585,
  38833,
  39166,
  39533,
  39741,
  40105,
  40294,
  40534,
  40730,
  41019,
  41328,
  43873,
  44506,
  45854,
  46826,
  48859,
  55268,
  62339,
]

BEIDOU_CAT_IDS = [
  36287,
  36590,
  36828,
  37210,
  37256,
  37384,
  37763,
  37948,
  38091,
  38250,
  38251,
  38775,
  38953,
  40549,
  40748,
  40749,
  40938,
  41434,
  41586,
  43001,
  43002,
  43107,
  43108,
  43207,
  43208,
  43245,
  43246,
  43539,
  43581,
  43582,
  43602,
  43603,
  43622,
  43623,
  43647,
  43648,
  43683,
  43706,
  43707,
  44204,
  44231,
  44337,
  44542,
  44543,
  44709,
  44793,
  44794,
  44864,
  44865,
  45344,
  45807,
  56564,
  58654,
  58655,
  61186,
  61187,
]

NORAD_CAT_IDS = QZSS_IDS + \
  GLONASS_IDS + \
  GALILEO_IDS + \
  BEIDOU_CAT_IDS + \
  GPS_IDS

BASE_URL = "https://www.space-track.org/basicspacedata/query/class/gp/norad_cat_id"

# Rate limit settings: max 30 requests per 1 minute and max 300 requests per hour
MAX_CALLS_PER_MINUTE = 30
MAX_CALLS_PER_HOUR = 300

session = requests.Session()

def authenticate():
    login_url = "https://www.space-track.org/ajaxauth/login"
    credentials = {
        "identity": SPACE_TRACK_USERNAME,
        "password": SPACE_TRACK_PASSWORD,
    }
    response = session.post(login_url, data=credentials)
    if response.status_code != 200:
        raise Exception("Authentication failed! Check your credentials.")

@sleep_and_retry
@limits(calls=MAX_CALLS_PER_MINUTE, period=60)
@limits(calls=MAX_CALLS_PER_HOUR, period=3600)
def fetch_omm(norad_cat_id):
    """
    Fetch the latest omm for a given NORAD CAT ID and apply rate limiting.
    """
    url = f"{BASE_URL}/{norad_cat_id}/format/json"
    response = session.get(url)
    if response.status_code == 200 and "error" not in response.text:
        return response.text.strip()
    else:
        print(f"No omm data found for NORAD CAT ID {norad_cat_id}: {response.text}")
        return None

def save_omm_to_file(filename, omm_data):
    with open(f"{OMM_DIR}/{filename}", "w") as file:
        file.write(omm_data)
    print(f"Saved omm to {filename}")

def download_gnss_omm():
    authenticate()
    for norad_cat_id in NORAD_CAT_IDS:
        filename = f"{norad_cat_id}.json"

        if os.path.isfile(f"{OMM_DIR}/{filename}"):
          continue

        omm_data = fetch_omm(norad_cat_id)
        if omm_data:
            save_omm_to_file(filename, omm_data)

download_gnss_omm()

omm_files = [f for f in os.listdir(OMM_DIR) if os.path.isfile(os.path.join(OMM_DIR, f))]
assert len(omm_files) == len(NORAD_CAT_IDS), f"Expected {len(NORAD_CAT_IDS)} omm files, but found {len(omm_files)}"

In [ ]:
import json
import pandas as pd

from zoneinfo import ZoneInfo
from datetime import datetime, timedelta
from skyfield.api import EarthSatellite, load, wgs84

observer = wgs84.latlon(LAT, LON)

local_tz = ZoneInfo(TIMEZONE)

start_time = datetime.now(local_tz).replace(hour=STAR_HOUR, minute=START_MINUTE, second=START_SECOND, microsecond=0)
end_time = datetime.now(local_tz).replace(hour=END_HOUR, minute=END_MINUTE, second=END_SECOND, microsecond=0)

time_difference = end_time - start_time
current_time = start_time + (time_difference / 2)

ts = load.timescale()

dfs = []
omm_files = [f for f in os.listdir(OMM_DIR) if f.endswith('.json')]

for omm_filename in omm_files:
    satellite_id = os.path.splitext(omm_filename)[0]
    omm_file_path = os.path.join(OMM_DIR, omm_filename)

    with open(omm_file_path, 'r') as omm_file:
        omm_data = json.load(omm_file)[0]

        satellite = EarthSatellite.from_omm(ts, omm_data)
        difference = satellite - observer

        t = ts.from_datetime(current_time)

        topocentric = difference.at(t)
        alt, az, distance = topocentric.altaz()

        if alt.degrees > 0:
          formatted_time = current_time.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'

          temp_df = pd.DataFrame({
              'time': [formatted_time],
              'satellite_id': [satellite_id],
              'altitude': [alt.degrees],
              'azimuth': [az.degrees],
          })

          dfs.append(temp_df)

df = pd.concat(dfs, ignore_index=True)

In [ ]:
df.head()

In [ ]:
def get_satellite_type(satellite_id):
    if satellite_id in QZSS_IDS:
        return 'QZSS'
    elif satellite_id in GALILEO_IDS:
        return 'Galileo'
    elif satellite_id in GLONASS_IDS:
        return 'GLONASS'
    elif satellite_id in GPS_IDS:
        return 'GPS'
    elif satellite_id in BEIDOU_CAT_IDS:
        return 'BeiDou'

df['satellite_id'] = df['satellite_id'].astype(int)
df['satellite_type'] = df['satellite_id'].apply(get_satellite_type)

df.head()

In [ ]:
df.to_csv(f"{OMM_DIR}/satellites.csv", index=False)